In [19]:
import os
from pypdf import PdfReader
import pandas as pd
import re
from abc import ABC, abstractmethod

In [65]:
class PageIdentifier(ABC):
    def __init__(self, base_dir, result_dir, verbose=False):
        self.base_dir = base_dir
        self.result_dir = result_dir
        self.verbose = verbose
        self.pages = {
            "Aktiva": [],
            "Passiva": [],
            "GuV": []
        }
        
    def walk_dir(self):
        for root, _, files in os.walk(self.base_dir):
            for file in files:
                if file.endswith(".pdf"):
                    filepath = os.path.join(root, file)
                    if self.verbose:
                        print(filepath)
                    
                    try:
                        self.identify_pages(filepath)
                    except Exception as e:
                        print(f"Error reading {filepath}: {e}")

    def __generate_dataframe(self):
        df_list = []
        for item in self.pages.keys():
            df = pd.DataFrame.from_records(self.pages[item], columns=['filepath', 'page'])
            df['type'] = item
            df_list.append(df)

        df = pd.concat(df_list, ignore_index=True)
        return(df)

    def save_to_csv(self):
        df = self.__generate_dataframe()
        df.to_csv(self.result_dir + "_identified_pages.csv", index=False)
        
    def get_dataframe(self):
        df = self.__generate_dataframe()
        return df

    def benchmark(self):
        df_truth = pd.read_csv("../Python/table_page_truth.csv")
        
    
    @abstractmethod
    def identify_pages(self, filepath):
        pass

In [66]:
class PageIdentifierRegex(PageIdentifier):
    def __init__(self, base_dir, result_dir, regex_patterns, verbose=False):
        super().__init__(base_dir, result_dir, verbose)
        self.regex_patterns = regex_patterns

    def identify_pages(self, filepath):
        reader = PdfReader(filepath)
        for page_num, page in enumerate(reader.pages):
            text = page.extract_text()

            for item in self.pages.keys():
                if all(
                    re.search(term, text, re.IGNORECASE | re.S) if isinstance(term, str) else False
                    for term in self.regex_patterns[item]
                ):
                    self.pages[item].append((filepath, page_num + 1))
                    if self.verbose:
                        print(f"Found {item} in {filepath} on page {page_num + 1}")


In [121]:
simple_regex_patterns = {
    "Aktiva": [
        r"aktiva",
        r"((20\d{2}).*(20\d{2}))"
    ],
    "Passiva": [
        r"passiva",
        r"((20\d{2}).*(20\d{2}))"
    ],
    "GuV": [
        r"gewinn",
        r"verlust",
        r"rechnung",
        r"((20\d{2}).*(20\d{2}))"
    ]
}

page_identifier_regex_1 = PageIdentifierRegex(
    # base_dir = "../Geschaeftsberichte/Berlin Energie und Netzholding/",
    base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/regex_1",
    regex_patterns = simple_regex_patterns,
    verbose = False
)

page_identifier_regex_1.walk_dir()

In [139]:
df1 = page_identifier_regex_1.get_dataframe()
df1

,filepath,page,type
0,../Geschaeftsberichte/Berliner Bäder Betriebe/...,10,Aktiva
1,../Geschaeftsberichte/Berliner Bäder Betriebe/...,15,Aktiva
2,../Geschaeftsberichte/Berliner Bäder Betriebe/...,22,Aktiva
3,../Geschaeftsberichte/Berliner Bäder Betriebe/...,28,Aktiva
4,../Geschaeftsberichte/Berliner Bäder Betriebe/...,11,Aktiva
...,...,...,...
793,../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch...,58,GuV
794,../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch...,62,GuV
795,../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch...,64,GuV
796,../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch...,66,GuV


In [135]:
df_truth = pd.read_csv("../Python/table_page_truth.csv")
df_truth = df_truth.assign(type=df_truth['type'].str.split('&')).explode('type')


In [140]:
def benchmark(df):
    df_temp = df_truth.join(
        df.set_index(['filepath', 'type']),
        on=['filepath', 'type'],
        how='left',
        rsuffix='_identified'
    )
    df_temp['filepath'] = df_temp['filepath'].str.replace(page_identifier_regex_1.base_dir, '', regex=False)
    df_temp['company'] = df_temp['filepath'].str.split('/').str[0]
    df_temp['file'] = df_temp['filepath'].str.split('/').str[1]
    df_temp = df_temp.drop(columns=['filepath'])

    correct_identified = df_temp[df_temp['page'] == df_temp['page_identified']]
    wrong_identified = df_temp[(df_temp['page'] != df_temp['page_identified']) & (df_temp['page_identified'].notna())]
    missing = df_temp[df_temp['page_identified'].isna()]

    print(f"Correctly identified: {len(correct_identified)}")
    print(f"Wrongly identified: {len(wrong_identified)}")
    print(f"Missing: {len(missing)}")

    return {'correct': correct_identified, 'wrong': wrong_identified, 'missing': missing}

In [143]:
ben_df1 = benchmark(df1)

Correctly identified: 199
Wrongly identified: 716
Missing: 25


In [144]:
ben_df1['missing']

,page,type,page_identified,company,file
62,16,GuV,NaN,IBB,ibb_digitaler_geschaeftsbericht_2017.pdf
63,17,GuV,NaN,IBB,ibb_digitaler_geschaeftsbericht_2017.pdf
166,18,Aktiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2011_BBB.pdf
167,19,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2011_BBB.pdf
169,28,Aktiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2018_BBB.pdf
170,29,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2018_BBB.pdf
175,32,Aktiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2016_BBB.pdf
176,33,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2016_BBB.pdf
178,26,Aktiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2009_BBB.pdf
179,27,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2009_BBB.pdf


In [132]:
text = PdfReader('../Geschaeftsberichte/IBB/ibb_digitaler_geschaeftsbericht_2017.pdf').pages[15].extract_text()
text

'16 // 19\nGewinn- und Verlustrechnung\nin Tsd. Euro Vorjahr\n1.  Zinserträge aus\n a)  Kredit- und Geldmarktgeschäften \ndarunter: verrechnete Aufwendungen aus Negativ-Zinsen\n b) festverzinslichen Wertpapieren und Schuldbuchforderungen\n234.656\n2.160\n57.034\n267.351\n758\n69.593\n291.690 336.994\n2.  Zinsaufwendungen 159.081 195.169\n darunter: verrechnete Erträge aus Negativ-Zinsen 11.207 7.750\n159.081 195.169\n132.609 141.775\n3. Laufende Erträge aus\n a) Aktien und anderen nicht festverzinslichen Wertpapieren\n b) Beteiligungen\n c) Anteilen an verbundenen Unternehmen \n0\n2\n0\n0\n2\n0\n2 2\n4.  Provisionserträge 3.508 3.829\n5.  Provisionsaufwendungen 817 777\n2.691 3.052\n6.  Sonstige betriebliche Erträge 7.073 89.293\n7.  Allgemeine Verwaltungsaufwendungen\n a) Personalaufwand\n  aa) Löhne und Gehälter\n  ab)  Soziale Abgaben und Aufwendungen  \nfür Altersversorgung und für Unterstützung\n   darunter: für Altersversorgung\n40.278\n9.178\n2.136\n41.037\n9.398\n2.560\n49.456 

In [107]:
re.search(r"passiva|passivseite", text, re.IGNORECASE | re.S)

<re.Match object; span=(1186, 1197), match='Passivseite'>

In [ ]:
regex_patterns_2 = {
    "Aktiva": [
        r"aktiva|aktivseite|a k t i v a|a k t i v s e i t e",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)"
    ],
    "Passiva": [
        r"passiva|passivseite|p a s s i v a|p a s s i v s e i t e",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)"
    ],
    "GuV": [
        r"gewinn",
        r"verlust",
        r"rechnung",
        r"((20\d{2}).*(20\d{2}))|vorjahr"
    ]
}

page_identifier_regex_2 = PageIdentifierRegex(
    # base_dir = "../Geschaeftsberichte/Berlin Energie und Netzholding/",
    base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/regex_2",
    regex_patterns = regex_patterns_2,
    verbose = False
)

page_identifier_regex_2.walk_dir()

KeyboardInterrupt: 

In [145]:
df2 = page_identifier_regex_2.get_dataframe()
ben_df2 = benchmark(df2)

Correctly identified: 218
Wrongly identified: 780
Missing: 21


In [146]:
ben_df2['missing']

,page,type,page_identified,company,file
169,28,Aktiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2018_BBB.pdf
170,29,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2018_BBB.pdf
175,32,Aktiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2016_BBB.pdf
176,33,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2016_BBB.pdf
178,26,Aktiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2009_BBB.pdf
179,27,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2009_BBB.pdf
181,24,Aktiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2014_BBB.pdf
182,25,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2014_BBB.pdf
184,16,Aktiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2007_BBB.pdf
185,17,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2007_BBB.pdf
